# Impacto das Tarifas Americanas nas Economias Parceiras

## 🚀 Projeto: "Impacto das Tarifas Americanas nas Economias Parceiras"

### 🎯 Objetivo:
O objetivo deste projeto é analisar o impacto das tarifas impostas pelos EUA em seus parceiros comerciais, e como esses impactos afetam a balança comercial (déficit/superávit) e a população dos países. Utilizando PySpark, vamos calcular a relação entre as tarifas, a população e a economia de cada país, e classificar os países em categorias de risco econômico.

### ✅ O que será implementado:

1. **Calcular o "Impacto Tarifa/População"**
Vamos adicionar uma coluna ao nosso conjunto de dados que mostre o impacto das tarifas em relação à população de cada país. Esse cálculo nos ajuda a entender se países menores estão sendo mais impactados proporcionalmente pelas tarifas dos EUA.

```python
from pyspark.sql.functions import col
```

# Calculando o impacto da tarifa sobre a população
```python
df = df.withColumn("Tariff Impact", col("Trump Tariffs Alleged") / col("Population"))
```
Classificar países em categorias de risco econômico Com base na análise do déficit/superávit da balança comercial dos EUA e o nível de tarifas impostas aos países, vamos classificar os países em diferentes categorias de risco econômico. Os países serão classificados como "High Risk" (alto risco), "Medium Risk" (médio risco) ou "Low Risk" (baixo risco).

# Classificando os países de acordo com o risco econômico

```python
df = df.withColumn("Risk Category", when(
    (col("US 2024 Deficit") < 0) & (col("Trump Tariffs Alleged") > 30), "High Risk"
).when(
    (col("US 2024 Deficit") < 0), "Medium Risk"
).otherwise("Low Risk"))
```
🔧 Como executar:
Pré-requisitos:

Ter o PySpark instalado e configurado.

Dados de tarifas dos EUA, déficit comercial e população dos países disponivel em Data/Csv/TariffCalculationsplusPopulation.csv.
ou no link https://www.kaggle.com/datasets/danielcalvoglez/us-tariffs-2025

Passos para execução:

Carregue esse Notebook em sua plataforma
. Pycharm, Databricks, Jupyter....

Execute a análise e visualize os resultados.

📊 Resultados Esperados:
Após rodar o código, você terá um DataFrame com informações sobre:

O impacto das tarifas sobre a população de cada país.

A classificação de risco econômico de cada país, com base no déficit comercial e nas tarifas impostas.

🔄 Possíveis Expansões:
Adicionar mais dados econômicos, como PIB, exportações e importações.

Implementar visualizações para melhor entender os impactos econômicos.

Analisar como as respostas dos países (ex: retaliações tarifárias) afetam as categorias de risco.


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import when
from pyspark.sql.functions import sum as _sum

In [3]:
spark = SparkSession.builder.appName("Spark").getOrCreate()

25/04/16 17:29:51 WARN Utils: Your hostname, prado-MS-7E05 resolves to a loopback address: 127.0.1.1; using 192.168.0.107 instead (on interface enp2s0)
25/04/16 17:29:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/16 17:29:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
read_options = {
    "header": True,
    "sep": ";",
    "inferSchema": True,
}

df1 = spark.read.csv("./Data/csv/TariffCalculationsplusPopulation.csv",**read_options)
df1.show(6)


+-----------+---------------+---------------+-------------------------------+---------------------+--------------+----------+
|    Country|US 2024 Deficit|US 2024 Exports|US 2024 Imports (Customs Basis)|Trump Tariffs Alleged|Trump Response|Population|
+-----------+---------------+---------------+-------------------------------+---------------------+--------------+----------+
|Afghanistan|          -11.1|           11.4|                           22.6|                  49%|           25%|  41454761|
|    Albania|           13.4|          141.7|                          128.3|                  10%|           10%|   2745972|
|    Algeria|      -1,447.10|       1,014.50|                       2,461.60|                  59%|           29%|  46164219|
|    Andorra|            1.5|            4.9|                            3.4|                  10%|           10%|     80856|
|     Angola|      -1,186.90|          682.4|                       1,869.20|                  63%|           32%|  36

In [5]:
df1.printSchema()

root
 |-- Country: string (nullable = true)
 |-- US 2024 Deficit: string (nullable = true)
 |-- US 2024 Exports: string (nullable = true)
 |-- US 2024 Imports (Customs Basis): string (nullable = true)
 |-- Trump Tariffs Alleged: string (nullable = true)
 |-- Trump Response: string (nullable = true)
 |-- Population: integer (nullable = true)



In [6]:
df1_clean = (
    df1
    .filter(col("Population").isNotNull()).filter(col("Trump Tariffs Alleged").isNotNull())
)

df1_clean = df1_clean.withColumn("US 2024 Deficit", regexp_replace(col("US 2024 Deficit"),",","").cast("float")) \
             .withColumn("US 2024 Exports",regexp_replace(col("US 2024 Exports"),",","").cast("float")) \
             .withColumn("US 2024 Imports (Customs Basis)",regexp_replace(col("US 2024 Imports (Customs Basis)"), ",", "").cast("float"))
display(df1_clean.show())

+-------------------+---------------+---------------+-------------------------------+---------------------+--------------+----------+
|            Country|US 2024 Deficit|US 2024 Exports|US 2024 Imports (Customs Basis)|Trump Tariffs Alleged|Trump Response|Population|
+-------------------+---------------+---------------+-------------------------------+---------------------+--------------+----------+
|        Afghanistan|          -11.1|           11.4|                           22.6|                  49%|           25%|  41454761|
|            Albania|           13.4|          141.7|                          128.3|                  10%|           10%|   2745972|
|            Algeria|        -1447.1|         1014.5|                         2461.6|                  59%|           29%|  46164219|
|            Andorra|            1.5|            4.9|                            3.4|                  10%|           10%|     80856|
|             Angola|        -1186.9|          682.4|         

None

In [7]:
df1_imports_in_decimal = df1_clean.withColumn("Trump Tariffs Alleged", regexp_replace(col("Trump Tariffs Alleged"),"%", "").cast("float")/ 100)
df1_imports_in_decimal.show(5)

+-----------+---------------+---------------+-------------------------------+---------------------+--------------+----------+
|    Country|US 2024 Deficit|US 2024 Exports|US 2024 Imports (Customs Basis)|Trump Tariffs Alleged|Trump Response|Population|
+-----------+---------------+---------------+-------------------------------+---------------------+--------------+----------+
|Afghanistan|          -11.1|           11.4|                           22.6|                 0.49|           25%|  41454761|
|    Albania|           13.4|          141.7|                          128.3|                  0.1|           10%|   2745972|
|    Algeria|        -1447.1|         1014.5|                         2461.6|                 0.59|           29%|  46164219|
|    Andorra|            1.5|            4.9|                            3.4|                  0.1|           10%|     80856|
|     Angola|        -1186.9|          682.4|                         1869.2|                 0.63|           32%|  36

In [8]:
df_Tarrif_impact_column = df1_imports_in_decimal.withColumn("Tarrif Impact",col("Trump Tariffs Alleged") / col("Population"))
display(df_Tarrif_impact_column.show(6))

+-------------------+---------------+---------------+-------------------------------+---------------------+--------------+----------+--------------------+
|            Country|US 2024 Deficit|US 2024 Exports|US 2024 Imports (Customs Basis)|Trump Tariffs Alleged|Trump Response|Population|       Tarrif Impact|
+-------------------+---------------+---------------+-------------------------------+---------------------+--------------+----------+--------------------+
|        Afghanistan|          -11.1|           11.4|                           22.6|                 0.49|           25%|  41454761|1.182011397918805...|
|            Albania|           13.4|          141.7|                          128.3|                  0.1|           10%|   2745972|3.641697730348307E-8|
|            Algeria|        -1447.1|         1014.5|                         2461.6|                 0.59|           29%|  46164219|1.278046098862844...|
|            Andorra|            1.5|            4.9|                 

None

In [9]:
df_Tarrif_impact_column.orderBy((col("Tarrif Impact")).desc()).select("Country","Tarrif Impact").show(10)

+--------------------+--------------------+
|             Country|       Tarrif Impact|
+--------------------+--------------------+
|               Nauru|4.968421052631579E-5|
|       Liechtenstein|1.831869510664993...|
|       Faroe Islands|1.817113909181013...|
|          Tuvalu (-)|1.018744906275468...|
|               Palau|5.641112427370678E-6|
|          San Marino|2.953337271116361...|
|           Gibraltar|2.599360557302903...|
|    Marshall Islands|2.575527339222706E-6|
|              Monaco|2.566998665160694...|
|British Virgin Is...|2.565089136847506E-6|
+--------------------+--------------------+
only showing top 10 rows



In [10]:
df1_risk_category = df_Tarrif_impact_column.withColumn("Risk Category",when((col("US 2024 Deficit") < 0 ) & (col("Trump Tariffs Alleged") * 100 > 30), "High Risk").when((col("US 2024 Deficit") < 0), "Medium Risk").otherwise("Low Risk"))

display(df1_risk_category.show(5))

+-----------+---------------+---------------+-------------------------------+---------------------+--------------+----------+--------------------+-------------+
|    Country|US 2024 Deficit|US 2024 Exports|US 2024 Imports (Customs Basis)|Trump Tariffs Alleged|Trump Response|Population|       Tarrif Impact|Risk Category|
+-----------+---------------+---------------+-------------------------------+---------------------+--------------+----------+--------------------+-------------+
|Afghanistan|          -11.1|           11.4|                           22.6|                 0.49|           25%|  41454761|1.182011397918805...|    High Risk|
|    Albania|           13.4|          141.7|                          128.3|                  0.1|           10%|   2745972|3.641697730348307E-8|     Low Risk|
|    Algeria|        -1447.1|         1014.5|                         2461.6|                 0.59|           29%|  46164219|1.278046098862844...|    High Risk|
|    Andorra|            1.5|     

None

In [11]:
savePath = "./Data/csv/clean/Tariff"
df1_risk_category.write.mode("overwrite").option("header", True).csv(savePath)

In [12]:
df1_High_Risk_category = df1_risk_category.select("Country","Risk Category","Population").where(col("Risk Category") == "High Risk")
display(df1_High_Risk_category.show())

+--------------------+-------------+----------+
|             Country|Risk Category|Population|
+--------------------+-------------+----------+
|         Afghanistan|    High Risk|  41454761|
|             Algeria|    High Risk|  46164219|
|              Angola|    High Risk|  36749906|
|          Bangladesh|    High Risk| 171466990|
|             Belarus|    High Risk|   9178298|
|Bosnia and Herzeg...|    High Risk|   3185073|
|            Botswana|    High Risk|   2480244|
|              Brunei|    High Risk|    458949|
|            Cambodia|    High Risk|  17423880|
|               China|    High Risk|1410710000|
|       Cote d'Ivoire|    High Risk|  31165654|
|      European Union|    High Risk| 448804042|
|       Faroe Islands|    High Risk|     54482|
|                Fiji|    High Risk|    924145|
|              Guyana|    High Risk|    826353|
|               India|    High Risk|1438069596|
|           Indonesia|    High Risk| 281190067|
|                Iraq|    High Risk|  45

None

In [13]:
df1_risk_category.groupBy("Risk Category").agg(_sum("Population").alias("Total Populations")).show()

+-------------+-----------------+
|Risk Category|Total Populations|
+-------------+-----------------+
|     Low Risk|       1542456975|
|  Medium Risk|        574588954|
|    High Risk|       5255849848|
+-------------+-----------------+



In [14]:
df1_high_exports =  df1_risk_category.filter((col("US 2024 Exports") > 10000) & (col("Risk Category") == "High Risk")).select("Country","Risk Category","Population","Tarrif Impact").orderBy(col("Tarrif Impact"))
display(df1_high_exports.show())

+--------------+-------------+----------+--------------------+
|       Country|Risk Category|Population|       Tarrif Impact|
+--------------+-------------+----------+--------------------+
|         India|    High Risk|1438069596|3.615958514430618...|
|         China|    High Risk|1410710000|4.749381517108406...|
|European Union|    High Risk| 448804042|8.689761310126526...|
|     Indonesia|    High Risk| 281190067|2.276040568673430...|
|      Mexico**|    High Risk| 129739759|2.620630735101026...|
|         Japan|    High Risk| 124516650|3.694285061475714...|
|       Vietnam|    High Risk| 100352192|8.968413963493693E-9|
|      Thailand|    High Risk|  71702435|1.004149998532127...|
|      Malaysia|    High Risk|  35126298|1.338028846649311E-8|
|       Israel*|    High Risk|   9756600|3.382325810220774E-8|
|   Switzerland|    High Risk|   8888093| 6.86311450611509E-8|
+--------------+-------------+----------+--------------------+



None